In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.tree import *
from sklearn.svm import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.model_selection import *
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.ensemble import *
import seaborn as sns
from sklearn.svm import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.gaussian_process import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.impute import *
import time

In [3]:
train = pd.read_csv('data/UNSW_NB15_training-set.csv',index_col='id')
train.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [4]:
test = pd.read_csv('data/UNSW_NB15_testing-set.csv',index_col='id')
test.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0


In [5]:
data = pd.concat([train,test])

In [6]:
data = data.replace({'-':np.nan})
data.loc[data.is_ftp_login>1,'is_ftp_login'] = 1
data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.121478,tcp,NaN,FIN,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
2,0.649902,tcp,NaN,FIN,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
3,1.623129,tcp,NaN,FIN,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
5,0.449454,tcp,NaN,FIN,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0


In [7]:
data['pkts']  = (data['spkts']  - data['dpkts'])
data['bytes'] = (data['sbytes'] - data['dbytes'])
data['loss']  = (data['sloss']  + data['dloss'])
data['inkpt'] = (data['sinpkt'] - data['dinpkt'])
data['ttl']   = (data['sttl']   + data['dttl'])
data['load']  = (data['sload']  - data['dload'])
data['jit']   = (data['sjit']   + data['djit'])
data['tcpb']  = (data['stcpb']  - data['dtcpb'])
data['mean']  = (data['smean']  - data['dmean'])

In [8]:
for i in set(set(data.service.unique()) - set(['dns','http','smtp','ftp-data','ftp'])):
        data.loc[data.service == i,"service"]= 'other_service'

for i in set(set(data.proto.unique()) - set(['tcp','udp','unas'])):
        data.loc[data.proto == i,"proto"]= 'other_proto'

for i in set(set(data.state.unique()) - set(['FIN','INT','CON'])):
        data.loc[data.state == i,"state"]= 'other_state'

In [10]:
def preprocess_initial(df): 
    dummies1 = pd.get_dummies(data['proto'],drop_first= True)
    dummies2 = pd.get_dummies(data['service'],drop_first= True)
    dummies3 = pd.get_dummies(data['state'],drop_first= True)
    
    df       = pd.concat([dummies1, dummies2, dummies3, df], axis=1)
    df       = df.drop(['attack_cat','proto','service','state'], axis=1)
    
    right = ['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', "rate",  "dttl",
           'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
            'djit',"swin", 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 
             'ackdat', 'smean', 'dmean','ct_srv_src', 'ct_dst_ltm', 
             'ct_src_dport_ltm', 'ct_dst_sport_ltm','ct_dst_src_ltm', 
             'ct_src_ltm', 'ct_srv_dst']
    
    left = ["sttl",'ct_state_ttl']
    
    for i in left:
        df[i] = df[i]**3
        #df[i] = df[i]**2
        
    for i in right:
        df[i] = (df[i]+1).transform(np.log)
        #df[i], _ = stats.boxcox(df[i]+1) 
        
    return df

data = preprocess_initial(data)

In [11]:
data.head(10)

,tcp,udp,unas,ftp,ftp-data,http,other_service,smtp,FIN,INT,other_state,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,pkts,bytes,loss,inkpt,ttl,load,jit,tcpb,mean
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,1,0,0,0.114647,1.945910,1.609438,5.556828,5.153292,4.318654,16003008,5.541264,9.558172,9.047394,0.000000,0.000000,3.230630,2.238047,3.439698,2.551833,5.545177,20.248085,21.512874,5.545177,0.000000,0.000000,0.000000,3.784190,3.784190,0,0,0.693147,0,0.693147,0.693147,0.693147,0.693147,0,0,0,0.693147,0.693147,0,0,2,86,0,15.920600,506,5663.577146,42.008151,-1580760939,0
2,1,0,0,0,0,0,0,0,1,0,0,0.500716,2.708050,3.663562,6.599870,10.645782,4.375422,238328,5.533389,9.035524,13.129483,1.098612,2.890372,3.930158,2.799283,4.133997,7.236180,5.545177,21.072432,21.847347,5.545177,0.000000,0.000000,0.000000,3.970292,7.009409,0,0,3.784190,1,0.693147,0.693147,0.693147,1.098612,0,0,0,0.693147,1.945910,0,0,-24,-41280,19,34.482135,314,-495176.200195,1449.205264,-1659503825,-1054
3,1,0,0,0,0,0,0,0,1,0,0,0.964368,2.197225,2.833213,5.899897,9.486987,2.719330,238328,5.533389,7.360913,11.017485,0.693147,1.945910,5.450504,4.641861,9.751535,9.343290,5.545177,21.472865,21.809507,5.545177,0.106068,0.059643,0.049208,3.850148,6.715383,0,0,2.079442,1,1.098612,0.693147,0.693147,1.386294,0,0,0,1.098612,1.945910,0,0,-8,-12822,7,129.138368,314,-59356.958619,28600.513090,-846964266,-778
4,1,0,0,1,0,0,0,0,1,0,0,0.986429,2.564949,2.564949,6.444131,6.647688,2.686289,238328,5.533389,7.916143,8.119584,0.693147,1.386294,5.036151,4.513447,5.560990,8.515749,5.545177,20.825027,20.769618,5.545177,0.000000,0.000000,0.000000,3.970292,4.174387,0,0,0.693147,1,1.098612,0.693147,0.693147,1.386294,1,1,0,1.098612,0.693147,0,0,0,-142,4,62.640821,314,-618.443115,5250.864841,59676287,-12
5,1,0,0,0,0,0,0,0,1,0,0,0.371187,2.397895,1.945910,6.282267,5.594711,3.537295,16387064,5.533389,9.055147,8.291060,1.098612,0.693147,3.886712,4.339375,7.790215,4.760523,5.545177,21.613680,21.404924,5.545177,0.120784,0.068730,0.055656,3.988984,3.828641,0,0,3.784190,1,1.098612,1.098612,0.693147,3.713572,0,0,0,1.098612,3.688879,0,0,4,266,3,-27.909269,506,4574.439209,2531.644634,458983359,8
6,1,0,0,0,0,0,0,0,1,0,0,0.322473,2.397895,1.945910,6.282267,5.594711,3.699275,16387064,5.533389,9.221580,8.457472,1.098612,0.693147,3.711833,3.974829,7.707391,4.425451,5.545177,22.105591,21.308850,5.545177,0.159508,0.112731,0.052216,3.988984,3.828641,0,0,3.784190,1,1.098612,1.098612,0.693147,3.713572,0,0,0,1.098612,3.688879,0,0,4,266,3,-12.312222,506,5402.890624,2306.280842,2188115112,8
7,1,0,0,0,0,0,0,0,1,0,0,0.492932,2.397895,2.197225,6.282267,5.872118,3.320820,16387064,5.533389,8.706289,8.267085,1.098612,0.693147,4.237980,4.408397,8.363536,4.791008,5.545177,21.303977,21.292651,5.545177,0.133951,0.066851,0.071577,3.988984,3.806662,0,0,3.784190,1,0.693147,0.693147,0.693147,3.713572,0,0,0,0.693147,3.688879,0,0,2,180,3,-12.869933,506,2147.199463,4406.251289,20128733,9
8,1,0,0,0,0,0,0,0,1,0,0,0.419752,2.397895,2.197225,6.282267,5.872118,3.514318,16387064,5.533389,8.906329,8.467109,1.098612,0.693147,4.039431,4.205500,8.235249,4.787180,5.545177,19.143317,19.571273,5.545177,0.110302,0.057509,0.055832,3.988984,3.806662,0,0,3.784190,1,1.386294,1.386294,0.693147,3.713572,0,0,0,1.386294,3.688879,0,0,2,180,3,-10.260141,506,2622.780274,3889.543359,-110020598,9
9,1,0,0,0,0,0,0,0,1,0,0,0.433667,2.397895,2.197225,6.282267,5.872118,3.475471,16387064,5.533389,8.866271,8.427054,1.098612,0.693147,4.114325,4.235685,8.309339,4.678528,5.545177,20.600075,21.950071,5.545177,0.112064,0.064038,0.051122,3.988984,3.806662,0,0,3.784190,1,1.386294,1.386294,0.693147,3.713572,0,0,0,1.386294,3.688879,0,0,2,180,3,-7

In [13]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Define the estimator and RFE selector
estimator = RandomForestClassifier()
selector = RFE(estimator, n_features_to_select=21)  # Select 21 features
selector = selector.fit(train_x, train_y)

# Create a sorted list of features and their rankings
feature_ranking = sorted([(rank, feature) for rank, feature in zip(selector.ranking_, train_x.columns)])

# Extract the top 21 features
top_21_features = [feature for rank, feature in feature_ranking if rank <= 21]

print("Top 21 Features:", top_21_features)


Top 21 Features: ['bytes', 'ct_dst_src_ltm', 'ct_srv_dst', 'ct_srv_src', 'ct_state_ttl', 'dbytes', 'dinpkt', 'dload', 'dttl', 'dur', 'load', 'mean', 'pkts', 'rate', 'sbytes', 'sload', 'smean', 'sttl', 'synack', 'tcprtt', 'ttl', 'ackdat', 'loss', 'dmean', 'sinpkt', 'inkpt', 'sjit', 'djit', 'INT', 'ct_src_ltm', 'sloss', 'jit', 'dtcpb', 'ct_dst_ltm', 'dpkts', 'stcpb', 'tcpb', 'ct_src_dport_ltm', 'spkts', 'ct_dst_sport_ltm', 'swin']


In [24]:
train_x = data[top_21_features]

train_y  = data['label']
#train_x  = data.drop(['label'],axis=1)

In [29]:
from sklearn.preprocessing import StandardScaler, QuantileTransformer

def preprocess_final(X_train, X_test, y_train, y_test):
    # Standard Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    print("SS")
    # Quantile Transformation
    quantile_transformer = QuantileTransformer(output_distribution='normal')
    X_train = quantile_transformer.fit_transform(X_train)
    X_test = quantile_transformer.fit_transform(X_test)
    print("QT")
    return X_train, X_test, y_train, y_test

In [15]:
def f_score(X_train, X_test, y_train, y_test):
    
    classifiers = [
        KNeighborsClassifier(n_neighbors=10),
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        GradientBoostingClassifier(),
        AdaBoostClassifier(),
        BaggingClassifier(),
        ExtraTreesClassifier(),
        MLPClassifier(max_iter=500),
        BernoulliNB(),
        PassiveAggressiveClassifier(),
        RidgeClassifierCV(),
        SGDClassifier(),
        LogisticRegression(max_iter=500),
    ]
    
    model = XGBClassifier(silent=True,
                      booster = 'gbtree',
                      objective='binary:logistic',
                      n_estimators=500, 
                      max_depth=10,
                      verbosity=0
                     )
    
    eval_set = [(X_test, y_test)]
    eval_metric = ["logloss"]
    
    model.fit(X_train, y_train,
              early_stopping_rounds=50, 
              eval_metric=eval_metric, 
              eval_set=eval_set)
    
    yxgb=model.predict(X_test)
    f = f1_score(y_true=y_test,y_pred=yxgb,average="weighted")
    
    yield 'XGBoost',f
    
    for clf in classifiers:
        clf.fit(X_train,y_train.ravel())
        y_pred = clf.predict(X_test)
        f = f1_score(y_true=y_test,y_pred=y_pred,average="weighted")
        
        yield clf.__class__.__name__ , round(f,4)

In [16]:
def category(X,y):
    
    d = []
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    fold = 0
    for train_ix, test_ix in kfold.split(X, y):
        fold+=1
        
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]

        X_train, X_test, y_train, y_test = preprocess_final(X_train, X_test, y_train, y_test)

        l=[]
        for class_name , score in f_score(X_train, X_test, y_train, y_test):
            l.append({'name':class_name,f"fold{fold}" : score})
            
        if len(d)==0:
            d = pd.DataFrame(l).set_index('name')
        else:
            d = pd.concat([d,pd.DataFrame(l).set_index('name')],axis=1)
        print(f"Completed Fold{fold}")
    return d

In [30]:
result = category(train_x.values,train_y.values)
result['mean'] = result.mean(axis=1)
result

SS
QT


/Users/niyatijain/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/niyatijain/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.53462
[1]	validation_0-logloss:0.42127
[2]	validation_0-logloss:0.35244
[3]	validation_0-logloss:0.31201
[4]	validation_0-logloss:0.27573
[5]	validation_0-logloss:0.24744
[6]	validation_0-logloss:0.22329
[7]	validation_0-logloss:0.20473
[8]	validation_0-logloss:0.19229
[9]	validation_0-logloss:0.18172
[10]	validation_0-logloss:0.17430
[11]	validation_0-logloss:0.16898
[12]	validation_0-logloss:0.16508
[13]	validation_0-logloss:0.16152
[14]	validation_0-logloss:0.15936
[15]	validation_0-logloss:0.15705
[16]	validation_0-logloss:0.15549
[17]	validation_0-logloss:0.15504
[18]	validation_0-logloss:0.15385
[19]	validation_0-logloss:0.15444
[20]	validation_0-logloss:0.15447
[21]	validation_0-logloss:0.15565
[22]	validation_0-logloss:0.15457
[23]	validation_0-logloss:0.15624
[24]	validation_0-logloss:0.15825
[25]	validation_0-logloss:0.15739
[26]	validation_0-logloss:0.15728
[27]	validation_0-logloss:0.15652
[28]	validation_0-logloss:0.15779
[29]	validation_0-loglos

/Users/niyatijain/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/niyatijain/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.

Completed Fold1
SS
QT


...

[0]	validation_0-logloss:0.49280
[1]	validation_0-logloss:0.39539
[2]	validation_0-logloss:0.33756
[3]	validation_0-logloss:0.30348
[4]	validation_0-logloss:0.28316
[5]	validation_0-logloss:0.27225
[6]	validation_0-logloss:0.27136
[7]	validation_0-logloss:0.27640
[8]	validation_0-logloss:0.28446
[9]	validation_0-logloss:0.29676
[10]	validation_0-logloss:0.31247
[11]	validation_0-logloss:0.32928
[12]	validation_0-logloss:0.34803
[13]	validation_0-logloss:0.36723
[14]	validation_0-logloss:0.38856
[15]	validation_0-logloss:0.41094
[16]	validation_0-logloss:0.43365
[17]	validation_0-logloss:0.45718
[18]	validation_0-logloss:0.47634
[19]	validation_0-logloss:0.49990
[20]	validation_0-logloss:0.49133
[21]	validation_0-logloss:0.48812
[22]	validation_0-logloss:0.51146
[23]	validation_0-logloss:0.53473
[24]	validation_0-logloss:0.53336
[25]	validation_0-logloss:0.55626
[26]	validation_0-logloss:0.55752
[27]	validation_0-logloss:0.57963
[28]	validation_0-logloss:0.58222
[29]	validation_0-loglos

/Users/niyatijain/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Completed Fold10


/Users/niyatijain/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,fold1,fold2,fold3,fold4,fold5,fold6,fold7,fold8,fold9,fold10,mean
name,,,,,,,,,,,
XGBoost,0.924734,0.930632,0.898792,0.933662,0.91814,0.928585,0.925041,0.851678,0.89344,0.845557,0.905026
KNeighborsClassifier,0.928300,0.927900,0.931000,0.929700,0.92900,0.931500,0.930400,0.929000,0.93030,0.930200,0.929730
DecisionTreeClassifier,0.924800,0.932000,0.918500,0.929400,0.91880,0.926400,0.925800,0.925000,0.92770,0.924500,0.925290
RandomForestClassifier,0.944100,0.948200,0.944300,0.946700,0.93700,0.947900,0.942200,0.942300,0.94300,0.942700,0.943840
GradientBoostingClassifier,0.927900,0.932900,0.929800,0.932200,0.92980,0.932800,0.932200,0.928200,0.93030,0.930200,0.930630
AdaBoostClassifier,0.918400,0.918600,0.921300,0.920600,0.91940,0.924000,0.922600,0.921800,0.91750,0.923100,0.920730
BaggingClassifier,0.940900,0.945000,0.935800,0.942300,0.93320,0.943600,0.940900,0.937700,0.93860,0.937000,0.939500
ExtraTreesClassifier,0.946600,0.946400,0.947900,0.947200,0.94520,0.947000,0.947300,0.944900,0.94460,0.945500,0.946260
MLPClassifier,0.899800,0.937600,0.917500,0.937800,0.93660,0.935200,0.899100,0.903700,0.90090,0.935800,0.920400
